# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-10 (started 2022-01-12)

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

## Modesty: A more complicated contract
* A donation to **Bob** that limits the amount that can be withdrawn in one transaction

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [ ]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

modest_pyteal = And (                                # Both conditions must be fulfilled
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= max_amount                       # Requested amount must be smaller than max_amount
)

# Security missing ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [ ]:
modest_teal = compileTeal(modest_pyteal, Mode.Signature, version=8)
print(modest_teal)

##### Step 3: Compile Teal -> Bytecode for AVM

In [ ]:
Modest = algod_client.compile(modest_teal)
Modest

#### Step 4: Deployment – Alice funds the smart signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modest['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

##### Step 5: Alice informs Bob

In [ ]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modest['result'])
print("Address of smart signature: ", Modest['hash'])

In [ ]:
# Check on Algoexplorer
print('https://testnet.algoexplorer.io/address/'+ Modest['hash'])

#### Step 6: Bob proposes a transaction that would withdraw too much

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- too much!!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6: Bob withdraws smaller amount

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- within limit!!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# Appendix: why the `Int(int(1e6))`
Pyteal requires the type `pyteal.Int` (with large I). This can only be obtained from Python's `int` type.

In [ ]:
a = 1E6
type(a)

In [ ]:
b = int(1E6)
type(b)

In [ ]:
c = Int(int(1E6))
type(c)

In [ ]:
# This is not possible
d = Int(1E6)